Ce fichier contient l'implémentation de notre modèle statique avec l'ensemble des contraintes imposées.

Pour utiliser ce fichier, il faut que le csv "dict_seatingX.csv" se trouve dans le même dossier. On a inclus les fichiers "dict_seatingX.csv" (X=1,...,9) dans le dossier et cela correspond aux 9 instances qui 
nous ont été proposées par l'équipe d'AirFrance, en respectant leur ordre sur l'Excel. Elles ont été créé grâce au notebook python "seating_extraction.ipynb"

In [2]:
using JuMP, Gurobi
using CSV, DataFrames
using BenchmarkTools

In [59]:
df_seating = CSV.read("dict_seating.csv")
poids = df_seating[!,:poids];
transit_time = df_seating[!,:transit_time]
group_members = df_seating[!,:group_members]
business = df_seating[!,:classe];
enfants = Array(df_seating[!,:enfant])
WCHR = Array(df_seating[!,:WCHR])
display(df_seating)


,Column1,genre,enfant,transit_time,classe,WCHR,group_members,poids
,Int64,Int64,Int64,Int64,Int64,Int64,String,Int64
1,1,1,0,0,1,0,"[1, 2]",80
2,2,0,0,0,1,0,"[1, 2]",70
3,3,1,0,0,1,0,"[3, 4]",80
4,4,0,0,0,1,0,"[3, 4]",70
5,5,0,0,0,1,0,[5],70
6,6,0,0,0,1,0,"[6, 7]",70
7,7,1,1,0,1,0,"[6, 7]",35
8,8,1,0,0,1,0,[8],80
9,9,0,0,0,0,0,"[9, 10]",70


# Choix de Type d'avion

In [82]:
avion_type = "A320" #A320 = 29 rangées // A321 = 36 rangées

if avion_type == "A320"
    secours = [11,12]
    Bc_y_min = 13.5
    Bc_y_max = 17.5
    nb_rangees = 29

elseif avion_type =="A321"
    secours = [16,20]
    Bc_y_min = 16.5
    Bc_y_max = 20.5
    nb_rangees = 36
end

nb_passagers = length(poids)
nb_colonnes = 6;
buss_pass = collect(i for i in 1:nb_passagers if business[i] ==1)
nb_business = length(buss_pass)
;

## Création d'une liste contenant tous les groupes et d'un dictionnaire qui donne le groupe de chaque passager

In [83]:
#création du dictionnaire contenant tous les membres du groupe
groups = Dict()
groups_unique = Dict()
for i = 1:nb_passagers
    group = group_members[i]
    group = replace(group, "[" => "")
    group = replace(group, "]" => "")
    group = replace(group, "," => "")
    a = [parse(Int, ss) for ss in split(group)]
    groups[i] = a
end


a = []
unique_group = []

nb_groups = 0

for i=1:nb_passagers
    if a != groups[i]
        a = groups[i]
        push!(unique_group,a)
        nb_groups += 1
    end
end


# Group d'enfants

Ici on résout le problème des groupes d'enfants. S'il existe plus que 2 enfants par adulte dans un groupe
leur placement dans la même demi-rangée avec un adulte de leur groupe devient impossible (cf. rapport final).
Dans ce cas, on les considère comme des passagers adultes (avec des poids d'enfants) et la distance aux adultes du groupe est minimisée.

In [84]:

for x in unique_group
    group_size = length(x)
    num_enfants = 0
    num_adults =0 
    for i in x
        if enfants[i] == 1
            num_enfants += 1
        else
            num_adults += 1
        end
    end
    if num_enfants/2 > num_adults
        for i in x
            enfants[i] = 0 
        end
    end
end

# Réinitialisation se fait d'ici à chaque fois

In [85]:
# Creation du modèle et des variables de décision.
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "Presolve" => 1, "OutputFlag" => 1,"Method"=>-1,)
set_time_limit_sec(model::Model, 60)
@variable(model, delta[1:nb_rangees,1:nb_colonnes,1:nb_passagers],Bin)

# la variable de décision gamma est seulement créée si des enfants sont présents sur l'avion
if 1 in enfants
    @variable(model,gamma[1:nb_rangees, 1:nb_colonnes, 1:nb_passagers] >= -1,Int)
end

# On définit le max et min rangée et colonne pour chaque groupe pour qu'ils soient réunis sur des rangées proches

@variable(model, 1 <= max_col[1:nb_groups] <= nb_colonnes+1,Int) #le +1 prend en compte l'allée centrale
@variable(model, 1 <= min_col[1:nb_groups] <= nb_colonnes+1,Int) 
@variable(model, 1 <= min_ran[1:nb_groups] <= nb_rangees,Int)
@variable(model, 1 <= max_ran[1:nb_groups] <= nb_rangees,Int)
;

#On assure que le max est plus grand que le min
for g=1:nb_groups
    @constraint(model, min_col[g] <= max_col[g])
    @constraint(model, min_ran[g] <= max_ran[g])
end

Academic license - for non-commercial use only


# Contraintes de base - contraintes physiques

In [86]:
#un passager par place
one_pass = Dict()
for i = 1:nb_rangees
    for j = 1:nb_colonnes
        one_pass[i+j] = @constraint(model, sum(delta[i,j,:]) <= 1)
    end
end


#chaque passager est attribué une place
every_pass = Dict()

for k = 1:nb_passagers
    every_pass[k] = @constraint(model, sum(delta[:,:,k]) == 1)
end

;

# Matrices qui permettent de trouver max et min des rangées / colonnes

In [87]:
#Ces matrices nous aident à assurer que tout les passagers se trouve dans les limites imposer par leur groupe et que 
#ces limites sont minimisé

#La matrice T est de la même taille de l'avion est vaut 1,2,3 pour des valeurs dans les premières trois colonnes
#Ça vaut 5,6,7 pour des valeurs dans les dernières trois colonnes respectivement
T = zeros(Int64,nb_rangees,nb_colonnes)
for j=1:nb_colonnes
    for i =1:nb_rangees
        if j<=3
            T[i,j] = j
        else 
            T[i,j]=j+1 #pour prendre en compte la présence de l'allée centrale et favoriser certaines configurations
        end
    end
end

R = zeros(Int64,nb_rangees,nb_colonnes)
for j=1:nb_colonnes
    for i =1:nb_rangees
        R[i,j] = i
    end
end

In [88]:
#Chaque passager doit être placé dans les places définiées par le min / max pour sa colonne / rangée
for g=1:nb_groups
    for k in unique_group[g]
        @constraint(model, sum(T.*delta[:,:,k]) <= max_col[g])
        @constraint(model, min_col[g] <= sum(T.*delta[:,:,k]))
    end
    
end

for g=1:nb_groups
    for k in unique_group[g]
        @constraint(model, sum(R.*delta[:,:,k]) <= max_ran[g])
        @constraint(model, min_ran[g] <= sum(R.*delta[:,:,k]))
    end
end

# Centrage de l'avion - position du barycentre des passagers

In [89]:
#w une matrice utilisée pour ajouter une distance de 1 pour les personnes qui se trouvent sur l'autre coté de l'allée centrale. 
#utilisé pour calculer la centre de mass dans la direction x -->
w = zeros(Int64,nb_rangees,nb_colonnes)
for i = 1:nb_rangees
    for j = 1:nb_colonnes
        if j >= 4
            w[i,j] = 1
        end
    end
end

#Poids total de tout les passagers
total_poids = 0
for k= 1:nb_passagers
    total_poids += poids[k]
end

#Centrage dans la direction X
m = 0 

for i = 1:nb_rangees
    for j = 1:nb_colonnes
        m += sum(delta[i,j,:].*poids)*(j-0.5 + w[i,j]) 
    end
end

x_G = m/total_poids

@constraint(model, 2.5 <= x_G <= 4.5)

#Centrage dans la direction Y

n = 0

for i = 1:nb_rangees
    for j = 1:nb_colonnes
            n += sum(delta[i,j,:].*poids)*(i-0.5)
    end
end

y_G = n/total_poids 

@constraint(model,Bc_y_min <= y_G <= Bc_y_max);

# Business Class

In [90]:
#On place les passagers business à l'avant de l'avion, en déterminant la zone qui leur ait réservée. 
#On suppose en effet qu'on peut délimiter la business class grâce à un rideau mobile
rang_bus_cab = nb_business ÷ 4
if nb_business % 4 != 0
    rang_bus_cab += 1
end
if rang_bus_cab >= 1
    for i=1:rang_bus_cab
        @constraint(model, sum(delta[i, j, k] for j=2 for k=1:nb_passagers) == 0)
        @constraint(model, sum(delta[i, j, k] for j=5 for k=1:nb_passagers) == 0)
    end
end 
                                
for k=1:nb_passagers
    if business[k]==1
        @constraint(model, sum(delta[1:rang_bus_cab,:,k]) >= 1)
    end
end

# Contrainte: Placement des enfants

In [91]:
# Les enfants ne doivent pas être isolés (présence d'un adulte de leur groupe à côté d'eux)
if 1 in enfants
    for k=1:nb_passagers
        if enfants[k]==1 && business[k]!=1
            for i=1:nb_rangees
                for j=1:nb_colonnes
                    @constraint(model, gamma[i, j, k] == -1*delta[i, j, k])
                end
            end
        else
            for i=1:nb_rangees
                for j=1:nb_colonnes
                    @constraint(model, gamma[i, j, k] == 3*delta[i, j, k])
                end
            end
        end
        
        if length(groups[k]) > 1
            for i=1:nb_rangees
                @constraint(model, sum(gamma[i, j, m] for j=1:3 for m in groups[k]) >= 0)
                @constraint(model, sum(gamma[i, j, m] for j=4:6 for m in groups[k]) >= 0)                                                                                     
            end
        end
    end
    
    for i in secours #on interdit que les enfants soient placés en face des issues de secours en imposant 
                        #une contrainte sur la variable de décision telle que ce soit nécessairement un adulte
        @constraint(model, sum(gamma[i, j, k] for j=6 for k=1:nb_passagers) >= 1) 
        @constraint(model, sum(gamma[i, j, k] for j=1 for k=1:nb_passagers) >= 1)
    end                                                                                                
end       
;   

## Placement des passagers en fauteuil roulant

On place les passagers en fauteuil roulant derrière la zone réservée aux passagers en classe business. Les places sur lesquelles seront affectées les passagers à mobilité réduite sont donc parfaitement définies.On suppose ici qu'il y a un maximum de 6 personnes à mobilité réduite sur chaque vol (c'est le cas dans les 9 instances que nous considérerons).

In [92]:
#On place les passagers en fauteuil roulant derrière la zone réservée aux passagers en classe business

wc = rang_bus_cab +1 # Le rangée dans laquelle les wheelchair commencent 
num_wchr = 0 
if 1 in WCHR
    for k=1:nb_passagers
        if WCHR[k] == 1
            num_wchr += 1
            if num_wchr == 1
                @constraint(model, sum(delta[wc,3,k]) == 1)
                @constraint(model, sum(delta[wc:wc+1,2:3,:]) <= 1)
            elseif num_wchr ==2
                @constraint(model, sum(delta[wc,4,k]) == 1)
                @constraint(model, sum(delta[wc:wc+1,4:5,:]) <= 1)
            elseif num_wchr ==3
                @constraint(model, sum(delta[wc+2,3,k]) == 1)
                @constraint(model, sum(delta[wc+2:wc+3,2:3,:]) <= 1)
            elseif num_wchr ==4
                @constraint(model, sum(delta[wc+2,4,k]) == 1)
                @constraint(model, sum(delta[wc+2:wc+3,4:5,:]) <= 1)
            elseif num_wchr ==5
                @constraint(model, sum(delta[wc+4,3,k]) == 1)
                @constraint(model, sum(delta[wc+4:wc+5,2:3,:]) <= 1)
            elseif num_wchr ==6
                @constraint(model, sum(delta[wc+4,4,k]) == 1)
                @constraint(model, sum(delta[wc+4:wc+5,4:5,:]) <= 1)
            end
        end
    end
end

# Fonction Objectif

In [93]:
#matrice avec des 1 dans le premier tiers

x = zeros(Int64,nb_rangees,nb_colonnes)
for i = 1:nb_rangees
    for j = 1:nb_colonnes
        if i <= 13 # change pour l'avion A321
            x[i,j] = 1
        end
    end
end

# maximiser le nombre de passagers qui ont transit time == 1 qui sont dans le premiers tiers

obj1 = 0
for k = 1:nb_passagers
    obj1 += sum(delta[:, :, k].*x)*transit_time[k]
end

    
#On ajoute une constante dans la fonction objectif pour qu'elle reste de signe positif, et donc que le "gap" 
#soit informatif
@objective(model, Min, 100 + 0.01*sum(max_col.-min_col) + 0.99*sum(max_ran.-min_ran)-obj1) 
;

In [94]:
optimize!(model)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 35331 rows, 56018 columns and 368088 nonzeros
Model fingerprint: 0xde11bf95
Variable types: 2 continuous, 56016 integer (27840 binary)
Coefficient statistics:
  Matrix range     [2e-03, 3e+01]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 33544 rows and 29192 columns
Presolve time: 0.65s
Presolved: 1787 rows, 26826 columns, 204580 nonzeros
Variable types: 2 continuous, 26824 integer (26488 binary)

Root relaxation: objective 8.900000e+01, 4368 iterations, 1.25 seconds
Total elapsed time = 10.73s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   89.00000    0  370          -   89.00000      -     -   12s
H    0     0                     242.9900000   89.00000  63.4%     -   14s


# Affichage des résultats - Placement des passagers

In [95]:

print("Objective value:")
println(objective_value(model))
x = value.(delta)
z = zeros(Int64,nb_rangees,nb_colonnes)

for i = 1:nb_rangees
    for j = 1:nb_colonnes
        for k = 1:nb_passagers
            if x[i,j,k] == 1
                z[i,j] = k
            end
        end
    end
end

convert(DataFrame, z)

Objective value:169.91999999999996


,x1,x2,x3,x4,x5,x6
,Int64,Int64,Int64,Int64,Int64,Int64
1,129,0,1,2,0,4
2,99,0,100,82,0,3
3,132,0,8,5,0,135
4,6,0,7,128,0,127
5,51,125,124,11,107,21
6,48,50,55,34,108,101
7,159,49,74,64,63,134
8,56,54,0,65,66,141
9,57,47,52,154,78,44


In [96]:
println("Tous les passagers en transit: ",)
trans_pass = collect(i for i in 1:nb_passagers if transit_time[i] ==1)
println(collect(i for i in 1:nb_passagers if transit_time[i] ==1))
num_trans = length(trans_pass)
                
println("Tout les passsagers business: ")
business_pass = collect(i for i in 1:nb_passagers if business[i] ==1) 
println(collect(i for i in 1:nb_passagers if business[i] ==1)) 
num_buss = length(business_pass)                                  
                                
println("Tous les enfants: ")
println(collect(i for i in 1:nb_passagers if enfants[i] ==1)) 

println("Tout les WCHR: ")
println(collect(i for i in 1:nb_passagers if WCHR[i] ==1))                                                

Tous les passagers en transit: 
[32, 63, 64, 65, 66, 67, 124, 125, 126, 130, 139]
Tout les passsagers business: 
[1, 2, 3, 4, 5, 6, 7, 8, 82, 99, 100, 127, 128]
Tous les enfants: 
[7, 14, 15, 16, 20, 29, 41, 66, 104, 105, 118, 128, 137, 138, 148, 151, 157]
Tout les WCHR: 
Int64[]


Prise en compte de l'objectif de mettre vers l'avant de l'avion (dans le premier "tiers") les passagers avec un tremps de correspondance inférieur à 1h30. 

En pratique les 10 premières rangées de l'avion sont généralement occuppées par la cabine business et les personnes à mobilité réduite, c'est en tout cas le choix que nous avons fait. Le premier "tiers" est donc défini comme étant l'ensemble des rangées correpsondant aux classes business et celles pour les personnes à mobilité réduite, auxquelles on rajoute le tiers des rangées de la cabine économique restante.

In [79]:
time_succ = 0
for i=1:10
    for j=1:nb_colonnes
        if z[i,j] in trans_pass
            time_succ += 1 #on dénombre les passagers en transit <=1h30 qui sont placés dans le 1er tiers
        end
    end
end
println("Passengers with transit time < 1:30:00 in first third: $time_succ/$num_trans")
println("Business passengers in business class cabin $num_buss/$num_buss")

Passengers with transit time < 1:30:00 in first third: 11/11
Business passengers in business class cabin 13/13


# Affichage des résultats - Placement des groupes

In [80]:
# on affiche un avion qui montre quelles places sont accordées à quels groupes 
q = zeros(Int64,nb_rangees,nb_colonnes)
for g in 1:nb_groups
    for i = 1:nb_rangees
        for j = 1:nb_colonnes
            if z[i,j] in unique_group[g]
                q[i,j] = g
            end
        end
    end
end
convert(DataFrame, q)

,x1,x2,x3,x4,x5,x6
,Int64,Int64,Int64,Int64,Int64,Int64
1,32,0,40,48,0,1
2,37,0,73,4,0,1
3,5,0,4,61,0,3
4,2,0,2,61,0,49
5,58,36,52,80,50,83
6,78,78,16,60,78,70
7,77,77,77,31,31,53
8,14,67,57,31,63,44
9,30,56,59,33,74,59


In [81]:
println(value.(x_G)) 
print(value.(y_G))


3.504005340453935
14.730974632843784

In [ ]:
#CSV.write("Optimal_solution_3(nuit)_passagers.csv", convert(DataFrame,z))
#CSV.write("Optimal_solution_3(nuit)_groupes.csv", convert(DataFrame,q)) 

#En utilisant Optimal_solution_1_groupes.csv dans "Solution_visualisation.ipynb" on peut visualiser avec des 
#couleurs l'affectation des groupes, c'est ce rendu visuel qu'on inclut dans le rapport

println("Passengers with transit time < 1:30:00 in first third: $time_succ/$num_trans")
println("Business passengers in business class cabin $num_buss/$num_buss")

# Calcul de la valeur de la satisfaction client associée à la configuration

In [ ]:
function get_client_number(i, j, delta) #fonction qui renvoie le numéro du client à la rangée i place j
    x = value.(delta)
    for k=1:size(delta)[3]
        if x[i,j,k]==1
            return k
        end
    end
    return 0 #pas de client assis sur cette chaise
end

In [ ]:
function cost(delta)
    cost = 0
    nb_rows, nb_columns = size(delta)[1], size(delta)[2]
    for i=1:nb_rows
        for j=1:nb_columns
            cost += satisfaction(i, j, delta)
        end
    end
    return cost
end

function satisfaction(i, j, delta)
    x = value.(delta)
    client = get_client_number(i, j, delta)
    if client==0
        return 0
    end
    group = groups[client]
    group_size = length(group)
    score = 0
    side = 1
    front = 0.5
    behind = 0.5
    alley = 1
    if group_size <= 3
        front = 0.25
        behind = 0.25
        alley = 0.25
    end
    if i==1
        if j==1
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==3
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += alley
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==4
            if get_client_number(i, j - 1, delta) in group
                score += alley
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==6
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        else
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        end
    elseif i==size(delta)[1]
        if j==1
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
        elseif j==3
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += alley
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
        elseif j==4
            if get_client_number(i, j - 1, delta) in group
                score += alley
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
        elseif j==6
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
        else
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
        end
    else
        if j==1
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==3
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += alley
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==4
            if get_client_number(i, j - 1, delta) in group
                score += alley
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        elseif j==6
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        else
            if get_client_number(i, j - 1, delta) in group
                score += side
            end
            if get_client_number(i, j + 1, delta) in group
                score += side
            end
            if get_client_number(i - 1, j, delta) in group
                score += front
            end
            if get_client_number(i + 1, j, delta) in group
                score += behind
            end
        end
    end
    return score
end

In [ ]:
@btime cost(delta)